# MaxFuse: Integrate snRNAseq with IMC lung tissue data (COVID-19 patients only) 

In [ ]:
import numpy as np
import pandas as pd
from scipy.io import mmread

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (6, 4)

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import anndata as ad
import scanpy as sc
import maxfuse as mf

import seaborn as sns

In [ ]:
# Set up output figure settings
plt.rcParams['figure.figsize']=(64,64) #rescale figures, increase sizehere

# Set up scanpy settings
sc.settings.verbosity = 3
sc.set_figure_params(dpi=100, dpi_save=300) #Increase DPI for better resolution figures

In [ ]:
import dill 

In [ ]:
#save the session
dill.dump_session('IMC_snRNAseq_maxfuse.db')

In [ ]:
#load the session
dill.load_session('COSMX_IPMN_maxfuse.db')

In [ ]:
import pickle 
with open('fusor.pkl', "wb") as f:
    pickle.dump(fusor, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle 
with open('fusor.pkl', 'rb') as f:
    fusor = pickle.load(f)

# 1- Import anndata objects

In [ ]:
pwd

Read in snRNAseq anndata object

In [ ]:
adata_10X = sc.read("./Iteration1/covid_lung_only_no_RBC.h5ad")

In [ ]:
adata_10X

In [ ]:
adata_10X.obs

In [ ]:
adata_10X.X.toarray()

In [ ]:
adata_10X.layers['counts'] = adata_10X.X.copy()
adata_10X.layers["scaled"] = sc.pp.scale(adata_10X, copy=True).X

In [ ]:
adata_10X.obs['fine_annotation']

In [ ]:
sc.pl.umap(adata_10X, color='fine_annotation') 

In [ ]:
# remmove erythrocytes - also removed in the IMC data
adata_10X = adata_10X[~adata_10X.obs['fine_annotation'].isin(['Erythrocytes']),:].copy()

In [ ]:
adata_10X.write('./covid_lung_only_no_RBC.h5ad')

Read in IMC anndata object

In [ ]:
adata_IMC = sc.read("./Iteration1/adata_COVID.h5ad")

In [ ]:
adata_IMC

In [ ]:
adata_IMC.obs

In [ ]:
sc.pl.umap(adata_IMC, color='cell_type') #remmove erythrocytes

In [ ]:
sc.pl.umap(adata_IMC, color='CD206')

In [ ]:
adata_IMC.X

In [ ]:
adata_IMC.var_names

In [ ]:
# check if IMC genes are in the snRNAseq data
sc.pl.umap(adata_10X, layer = 'counts',color = 'KRT7')

## Data preprocessing

In [ ]:
correspondence = pd.read_csv("./protein_gene_conversion.csv")
correspondence.head()

In [ ]:
rna_protein_correspondence = []

for i in range(correspondence.shape[0]):
    curr_protein_name, curr_rna_names = correspondence.iloc[i]
    if curr_protein_name not in adata_IMC.var_names:
        continue
    if curr_rna_names.find('Ignore') != -1: # some correspondence ignored eg. protein isoform to one gene
        continue
    curr_rna_names = curr_rna_names.split('/') # eg. one protein to multiple genes
    for r in curr_rna_names:
        if r in adata_10X.var_names:
            rna_protein_correspondence.append([r, curr_protein_name])
            
rna_protein_correspondence = np.array(rna_protein_correspondence)

In [ ]:
rna_protein_correspondence

In [ ]:
# Columns rna_shared and protein_shared are matched.
# One may encounter "Variable names are not unique" warning,
# this is fine and is because one RNA may encode multiple proteins and vice versa.
rna_shared = adata_10X[:, rna_protein_correspondence[:, 0]].copy()
protein_shared = adata_IMC[:, rna_protein_correspondence[:, 1]].copy()

In [ ]:
rna_shared.shape

In [ ]:
protein_shared.shape

Retrieve the shared features.
In this case we only use RNA or Protein features that are variable (larger than a certain threshold).

In [ ]:
# Make sure no column is static
mask = (
    (rna_shared.X.toarray().std(axis=0) > 0.3) 
    & (protein_shared.X.std(axis=0) > 0.1)
)
rna_shared = rna_shared[:, mask].copy()
protein_shared = protein_shared[:, mask].copy()
print([rna_shared.shape,protein_shared.shape])

In [ ]:
protein_shared.var_names

In [ ]:
# The following features were shared between IMC and snRNAseq data:
#'CD11b', 'CD3', 'CD3', 'CD3', 'CD4', 'CD8', 'CD45', 'CD11c', 'CD44',
#'CD14', 'CD45RO', 'CD31', 'CD107a', 'CD206', 'ICAM1', 'CD68', 'CD163',
#'CD38', 'CD16', 'Ki67', 'Vimentin', 'SMA', 'CD74', 'Collagen1',
#'GranzymeB', 'Iba1', 'MHCI', 'MHCI', 'MHCI', 'MHCII', 'MHCII', 'PanCK',
#'PanCK', 'PanCK', 'PanCK', 'Vista', 'vWF'

In [ ]:
# Process rna_shared
#sc.pp.normalize_total(rna_shared) - already done, skip this
#sc.pp.log1p(rna_shared) - already done, skip thi
sc.pp.scale(rna_shared)
sc.pp.scale(protein_shared)

In [ ]:
rna_shared = rna_shared.X.copy()
protein_shared = protein_shared.X.copy()

In [ ]:
rna_shared

In [ ]:
protein_shared

Apply standard preprocessing steps to **all available RNA measurements and protein measurements** (not just the shared ones) to get two arrays, `rna_active` and `protein_active`, which are used for iterative refinement. Again if the input data is already processed, these steps can be skipped.

In [ ]:
# process all RNA features
sc.pp.highly_variable_genes(adata_10X, n_top_genes=8000)
# only retain highly variable genes
adata_10X = adata_10X[:, adata_10X.var.highly_variable].copy()
sc.pp.scale(adata_10X)

In [ ]:
sc.pp.scale(adata_IMC)

In [ ]:
# make sure no feature is static
rna_active = adata_10X.X
protein_active = adata_IMC.X
rna_active = rna_active[:, rna_active.std(axis=0) > 1e-5] # these are fine since already using variable features
protein_active = protein_active[:, protein_active.std(axis=0) > 1e-5] # protein are generally variable

In [ ]:
# inspect shape of the four matrices
print(rna_active.shape)
print(protein_active.shape)
print(rna_shared.shape)
print(protein_shared.shape)

In [ ]:
rna_active

In [ ]:
protein_active

## Fitting MaxFuse

### Step I: preparations

In [ ]:
labels1 = adata_IMC.obs['pheno_cluster_new']
labels2 = adata_10X.obs['fine_annotation']

In [ ]:
labels1 = np.array(labels1)
labels2 = np.array(labels2)

In [ ]:
labels1

In [ ]:
labels2

In [ ]:
# call constructor for Fusor object
# which is the main object for running MaxFuse pipeline
fusor = mf.model.Fusor(
    shared_arr1=protein_shared,
    shared_arr2=rna_shared,
    active_arr1=protein_active,
    active_arr2=rna_active,
    labels1=labels1, #optional cluster labels for the rna dataset
    labels2=labels2 #optional cluster labels for the protein dataset
)

In [ ]:
fusor

To reduce computational complexity, call `split_into_batches` to fit the batched version of MaxFuse.

In [ ]:
fusor.split_into_batches(
    max_outward_size=8000,
    matching_ratio=2,
    metacell_size=2,
    verbose=True
)

In [ ]:
# plot top singular values of avtive_arr1 on a random batch
fusor.plot_singular_values(
    target='active_arr1',
    n_components=None # can also explicitly specify the number of components
)

In [ ]:
# plot top singular values of avtive_arr2 on a random batch
fusor.plot_singular_values(
    target='active_arr2',
    n_components=None
)

Inspect the "elbows" to choose the number of PCs to be used for both active data.
Then call `construct_graphs` to compute nearest-neighbor graphs as needed.

In [ ]:
fusor.construct_graphs(
    n_neighbors1=15,
    n_neighbors2=15,
    svd_components1=30,
    svd_components2=40,
    resolution1=2, #leiden clustering resolution
    resolution2=2, #leiden clustering resolution
    resolution_tol=0.1,
    verbose=True
)

### Step II: finding initial pivots

Use the shared arrays whose columns are matched to find initial pivots.
Plot top singular values of two shared arrays to determine how many PCs to use.

In [ ]:
# plot top singular values of shared_arr1 on a random batch
fusor.plot_singular_values(
    target='shared_arr1',
    n_components=None,
)

In [ ]:
# plot top singular values of shared_arr2 on a random batch
fusor.plot_singular_values(
    target='shared_arr2',
    n_components=None
)

In [ ]:
# In the case of integration that involves **spatial** data, 
# the developers encourage using ``wt1 = 0.3`` and ``wt2 = 0.3`` since such datasets are usually "weaker" in linkage and requires stronger "smoothing".
fusor.find_initial_pivots(
    wt1=0.3, wt2=0.3,
    svd_components1=28, svd_components2=30
)

Now, the set of *initial pivots* store the matched pairs when only the information in the shared arrays is used. The information on initial pivots are stored in the internal field ``fusor._init_matching`` that is invisible to users.

### Step III: finding refined pivots

Use the information in active arrays to iteratively refine initial pivots.
Plot the top canonical correlations to choose the best number of components in canonical correlation analysis (CCA).

In [ ]:
# plot top canonical correlations in a random batch
fusor.plot_canonical_correlations(
    svd_components1=None,
    svd_components2=50,
    cca_components=39
)

A quick check on the previous plot gives a rough guide on what the `refine_pivots` parameters should be picked.

**Note:** here that the `n_iters` number we choosed *1*, as in low snr datasets (eg. Spatial-omis) increase amount of iteration might degrade the performance.

In [ ]:
fusor.refine_pivots(
    wt1=0.3, wt2=0.3,
    svd_components1=None, svd_components2=40,
    cca_components=30,
    n_iters=1,
    randomized_svd=False, 
    svd_runs=1,
    verbose=True
)

**Note:** here we can see `filter_prop` we increased the pivot filtering to *0.5* compared to previous example. We found harsher filtering for integrations that involves spatial-omics is more beneficial.

In [ ]:
fusor.filter_bad_matches(target='pivot', filter_prop=0.5)

**Optional:** quick check the performance based on cell type accuracy (pivot matching).

In [ ]:
pivot_matching = fusor.get_matching(order=(1, 2),target='pivot')

lv1_acc = mf.metrics.get_matching_acc(matching=pivot_matching, 
    labels1=labels1, 
    labels2=labels2,
    order = (1, 2)
)
lv1_acc

In [ ]:
matched_pivot_cells = pd.DataFrame(list(zip(pivot_matching[0], pivot_matching[1], pivot_matching[2])), 
             columns = ['mod1_indx', 'mod2_indx', 'score'])
matched_pivot_cells.to_csv('MaxFuse_IMC_snRNAseq_matched_pivots.csv', index=True)

In [ ]:
matched_pivot_cells

In [ ]:
# We can inspect the first pivot pair.
[pivot_matching[0][0], pivot_matching[1][0], pivot_matching[2][0]]

In [ ]:
# compute the confusion matrix to see where the pivot matching goes wrong.
cm = confusion_matrix(labels1[pivot_matching[0]], labels2[pivot_matching[1]])
ConfusionMatrixDisplay(
    confusion_matrix=np.round((cm.T/np.sum(cm, axis=1)).T*100), 
    display_labels=np.unique(labels1)
).plot()

**Note:** With the `refine_pivots`, we can theoratically co-embed the *full* dataset into CCA space
. **However**, in the case that invovles *low-snr* datasets (eg. spatial-omics), the developers do not suggest projecting all the cells into a common space without any filtering steps. 
This is done after the `propogation` step.

### Step IV: propagation

Refined pivots can only give us a pivot matching that captures a subset of cells. In order to get a *full* matching that involves all cells during input, we need to call `propagate`.


In [ ]:
fusor.propagate(
    svd_components1=38, 
    svd_components2=None, 
    wt1=0.7,
    wt2=0.7,
)

`filter_bad_matches` with `target=propagated` to optionally filter away a few matched pairs from propagation. 

**Note:** In the best scenario, we would prefer all cells in the *full* dataset can be matched accross modality. However, in the case that invovles *low-snr* datasets (eg. spatial-omics), many cells are noisy (or lack of information) and should not be included in downstream cross-modality analysis.
The developers suggest in such scenarios, `filter_prop` should be set around *0.1-0.4*.

In [ ]:
fusor.filter_bad_matches(
    target='propagated',
    filter_prop=0.3
)

We use `get_matching` with `target='full_data'` to extract the full matching.

Because of the batching operation, the resulting matching may contain duplicates. The `order` argument determines how those duplicates are dealt with. 
`order=None` means doing nothing and returning the matching with potential duplicates;
`order=(1, 2)` means returning a matching where each cell in the first modality contains *at least one match* in the second modality;
`order=(2, 1)` means returning a matching where each cell in the second modality contains *at least one match* in the first modality.

**Note:** Since we filtered out some cell pairs here, not all cells in the full dataset has matches.

In [ ]:
full_matching = fusor.get_matching(order=(1, 2), target='full_data')

In [ ]:
full_matching

In [ ]:
matched_cells = pd.DataFrame(list(zip(full_matching[0], full_matching[1], full_matching[2])), 
             columns = ['mod1_indx', 'mod2_indx', 'score'])
# columns: cell idx in mod1, cell idx in mod2, and matching scores

In [ ]:
matched_cells

In [ ]:
#compression_opts = dict(method='zip', archive_name='MaxFuse_Cosmx1K_Cosmx6K_matched_pairs.csv') 
matched_cells.to_csv('MaxFuse_IMC_snRNAseq_matched_pairs.csv', index=True, chunksize=537862)

In [ ]:
# compute the cell type level matching accuracy, for the full (filtered version) dataset
lv1_acc = mf.metrics.get_matching_acc(matching=full_matching, 
    labels1=labels1, 
    labels2=labels2 
)
lv1_acc

In [ ]:
cm = confusion_matrix(labels1[full_matching[0]], labels2[full_matching[1]])
ConfusionMatrixDisplay(
    confusion_matrix=np.round((cm.T/np.sum(cm, axis=1)).T*100), 
    display_labels=np.unique(labels1)
).plot()

### Step V: downstream analysis

As mentioned before, we want to draw a UMAP in the joint-embedding space, but for the filtered version of cells. For the *low-snr* datasets (eg. spatial-omics), the developers suggest only using cells that passed the propogation filtering step.

In [ ]:
protein_cca, rna_cca_sub = fusor.get_embedding(
    active_arr1=fusor.active_arr1,
    active_arr2=fusor.active_arr2[full_matching[1],:] # cells in cosmx6k remained after filtering
)

In [ ]:
print(protein_cca.shape)

In [ ]:
print(rna_cca_sub.shape)

In [ ]:
# cells from IMC dataset in the cca embedding
protein_cca

In [ ]:
full_matching[0]

In [ ]:
rna_cca_sub

In [ ]:
full_matching[1]

In [ ]:
all_indexes = np.concatenate((full_matching[0], full_matching[1]), axis=0)

In [ ]:
all_indexes

In [ ]:
print(all_indexes.shape)

In [ ]:
dim_use = 15
allcca = np.concatenate((protein_cca[:,:dim_use], rna_cca_sub[randix,:dim_use]), axis=0)
allcca_df = pd.DataFrame(allcca)
allcca_df
allcca_df.to_csv("./IMC_snRNAseq_cca_preUmap.csv", index = True)

In [ ]:
allcca_df2 = pd.DataFrame(allcca, index=all_indexes)
allcca_df2 

In [ ]:
allcca_df2.to_csv("./IMC_snRNAseq_cca_preUmap_v2.csv", index = True)

Since we have *~170,000* cells for CODEX, calculating UMAP for the full dataset will take a while, for showcasing we can just subsample a smaller sample.

In [ ]:
np.random.seed(56)
subs = 50000
randix = np.random.choice(rna_cca_sub.shape[0],subs, replace = False)

dim_use = 15 # dimensions of the CCA embedding to be used for UMAP etc

cca_adata = ad.AnnData(
    np.concatenate((protein_cca[:,:dim_use], rna_cca_sub[randix,:dim_use]), axis=0), 
    dtype=np.float32
)
cca_adata.obs['data_type'] = ['IMC'] * protein_cca.shape[0] + ['snRNAseq'] * subs
cca_adata.obs['cell_type'] = list(np.concatenate((labels1,
                                                  labels2[full_matching[1]][randix]), axis = 0))

In [ ]:
sc.pp.neighbors(cca_adata, n_neighbors=15)
sc.tl.umap(cca_adata)


Diagnostic plots to QC the match of the snRNAseq data with the IMC data

In [ ]:
sc.pl.umap(cca_adata, color='data_type', save='cca_adata_data_type.pdf')

In [ ]:
sc.pl.umap(cca_adata, color='cell_type', save='cca_adata_cell_type.pdf')

In [ ]:
sc.pl.umap(cca_adata, color='cell_type', groups = ['AT1', 'AT2', 'AT2 cell',
                                                 'SARSCoV2+ AT2 cell', 'SARSCoV2+ Epithelial cell'], save= 'cca_adata_epithelial.pdf')


In [ ]:
sc.pl.umap(cca_adata, color='cell_type', groups = ['Interstitial Macrophage', 'Interstitial macrophages', 'Monocyte-derived macrophages',
                                                 'Proliferating Interstitial Macrophage', 'SARSCoV2+ Interstitial Macrophage', ], save= 'cca_adata_interstitial_macrophages.pdf')


In [ ]:
sc.pl.umap(cca_adata, color='cell_type', groups = ['Alveolar Macrophage', 'Alveolar macrophages', 'Apoptotic Alveolar Macrophage',
                                                 'Apoptotic SARSCoV2+ Alveolar Macrophage'], save= 'cca_adata_alveolar_macrophages.pdf')

In [ ]:
sc.pl.umap(cca_adata, color='cell_type', groups = ['ArginaseHighVISTAHigh Activated Neutrophil', 'SARSCoV2+ ArginaseHighVISTAHigh Activated Neutrophil','ArginaseLowVISTALow Activated Neutrophil', 'ArginaseLowVISTALow Neutrophil',
                                                 'CD16+ Neutrophils', 'CD16- Neutrophils'], save= 'cca_adata_neutrophils.pdf')


In [ ]:
cca_adata #anndata object based on the 15 CCA embeddings

In [ ]:
cca_adata.obs

In [ ]:
cca_adata.write('./cca_adata.h5ad')

In [ ]:
all_umap = cca_adata.obsm["X_umap"]
# convert to data a frame
all_umap = pd.DataFrame(all_umap, index=cca_adata.obs_names)
all_umap.to_csv("./IMC_snRNAseq_cca_umap.csv", index=True)  

In [ ]:
all_umap